In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score, fbeta_score, precision_score, recall_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import multilabel_confusion_matrix, plot_confusion_matrix, classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from skmultilearn.adapt import MLkNN
from sklearn.linear_model import LogisticRegression, Perceptron, RidgeClassifier, SGDClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.calibration import CalibratedClassifierCV
from sklearn.multioutput import MultiOutputClassifier


import pandas as pd
import json
import os
import re

import pickle
import nltk
from nltk.tokenize import word_tokenize

import gensim 
from gensim.models import Word2Vec
import gensim.downloader


import spacy
import matplotlib
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ast import literal_eval
from tqdm import tqdm

import sklearn.metrics
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('seaborn')

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [2]:
nltk.download('punkt')
tqdm.pandas()

[nltk_data] Downloading package punkt to /homes/lgf21/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [58]:
# Open Pickle without lemmatisation: 

with open('merged_data_no_duplicates.pickle', 'rb') as handle:
    (X_train_text, X_test_text, Y_train_tactic, Y_test_tactic, Y_train, Y_test) = pickle.load(handle)

In [18]:
X_train_text

0       Exploit Public-Facing Application - Enterprise...
1       Emergency Incident ResponseReport a Confirmed ...
2       Pass the Hash - Enterprise | MITRE ATT&CK\xe2\...
3       Extra Window Memory Injection - Enterprise | M...
4       Tropic Trooper Targets Taiwanese Government an...
                              ...                        
2142     Molerats Delivers  MALWARE_NAME  Backdoor to ...
2143     Transparent Tribe  Evolution analysis  part  ...
2144     WWW FIDELISSECURITY COM  Fidelis Cybersecurit...
2145     OilRig uses  MALWARE_NAME  IIS Backdoor on Ta...
2146     The OilRig Campaign  Attacks on Saudi Arabian...
Name: Text, Length: 2072, dtype: object

# Feature Extraction:

In [4]:
# call methods for feature extraction and evaluation:
import sys
sys.path.append('../src')

from methods import feature_extraction, evaluation

In [26]:
X_train, X_test = feature_extraction('TfIdfVectorizer', X_train_text, X_test_text)

/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [6]:
# Train and test: First delete techniques less than 9 
# We fix the random state to have the same dataset in our different tests@

sv_classifier = OneVsRestClassifier(LinearSVC(penalty = 'l2', loss = 'squared_hinge', dual = False, max_iter = 1000, class_weight = 'balanced', random_state=42), n_jobs = 1)
sv_classifier.fit(X_train, Y_train)


OneVsRestClassifier(estimator=LinearSVC(class_weight='balanced', dual=False,
                                        random_state=42),
                    n_jobs=1)

In [7]:
Y_pred = pd.DataFrame(sv_classifier.predict(X_test), columns=Y_test.columns)

In [8]:
evaluation(Y_pred, Y_test)

/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


,metric,result
0,macro precision,0.523972
1,micro precision,0.544892
2,macro recall,0.364063
3,micro recall,0.404830
4,macro fscore,0.452901
5,micro fscore,0.509628


In [84]:
# Save as pickle technique performance: 

#with open('postprocessing.pickle', 'wb') as handle:
 #   pickle.dump(sv_classifier, handle, protocol=pickle.HIGHEST_PROTOCOL)

OSError: [Errno 122] Disk quota exceeded

# Open Tactic Performance: 

In [19]:
# Open Pickle: 

with open('tactic_SVmodel.pickle', 'rb') as handle:
    tactic_model = pickle.load(handle)

In [20]:
data = '../src/scraping/scraped_data/tactic_dataset.json'

In [21]:
with open(data) as file:
    tactic_data = json.load(file)

In [27]:
Y_test_pred_tactic = pd.DataFrame(tactic_model.predict(X_test), columns = Y_test_tactic.columns)

# Hanging Node:

In [28]:
# list of tactics and list of techniques:
from sklearn.preprocessing import MinMaxScaler

In [29]:
tactic_prob = pd.DataFrame(tactic_model.decision_function(X_test), columns=Y_test_tactic.columns)

In [30]:
# if scaling at (-1,1) then threshold of .5 
# scaling determines improvement 
# (0-1) performs poorly with threshold of .5 

technique_prob = pd.DataFrame(MinMaxScaler((-1, 1)).fit_transform(sv_classifier.decision_function(X_test)), columns=Y_test.columns)

In [31]:
def hanging_node(i, tactic_proba, technique_proba,tactic_threshold=0.5, technique_threshold=0.5, a = 1,b = 0, c=1, d=0):
    assert a > technique_threshold and c > technique_threshold and b < tactic_threshold and d < tactic_threshold
    pred_tactics = [tactic for tactic in tactic_proba.columns if tactic_proba.iloc[i][tactic] > tactic_threshold]
    pred_techniques = [technique for technique in technique_proba.columns if technique_proba.iloc[i][technique] > technique_threshold]
    #return pred_tactics, pred_techniques
    print((pred_tactics, pred_techniques))
    for tactic in tactic_proba.columns:
        for technique in tactic_data[tactic]['Technique_ID'][0]:
            if technique in pred_techniques and tactic not in pred_tactics:
                if technique_proba.iloc[i][technique] > a and tactic_proba.iloc[i][tactic] > b:
                    pred_tactics.append(tactic)
                elif technique_proba.iloc[i][technique] < c and tactic_proba.iloc[i][tactic] < d:
                    pred_techniques.remove(technique)
    print(pred_tactics, pred_techniques)
    print('-----------------------------')
    return pred_tactics, pred_techniques
    

In [32]:
Y_test_tactics_hanging_node = []
Y_test_techniques_hanging_node = []
for i in range(len(X_test)):
    pred_tactics, pred_techniques = hanging_node(i, tactic_prob, technique_prob,tactic_threshold=0, technique_threshold=0.5,  a = 0.55, b = -0.9, c = 0.95, d = -0.30)
    Y_test_tactics_hanging_node.append([1 if tactic in pred_tactics else 0 for tactic in Y_test_tactic.columns])
    Y_test_techniques_hanging_node.append([1 if technique in pred_techniques else 0 for technique in Y_test.columns])

Y_test_tactics_hanging_node = pd.DataFrame(Y_test_tactics_hanging_node, columns=Y_test_tactic.columns)
Y_test_techniques_hanging_node = pd.DataFrame(Y_test_techniques_hanging_node, columns=Y_test.columns)

([], [])
[] []
-----------------------------
(['TA0040'], ['T1109', 'T1500'])
['TA0040'] ['T1109', 'T1500']
-----------------------------
(['TA0002', 'TA0003', 'TA0004', 'TA0005', 'TA0007'], ['T1082', 'T1059', 'T1060'])
['TA0002', 'TA0003', 'TA0004', 'TA0005', 'TA0007'] ['T1082', 'T1059', 'T1060']
-----------------------------
(['TA0004', 'TA0005'], ['T1134', 'T1111'])
['TA0004', 'TA0005', 'TA0006'] ['T1134', 'T1111']
-----------------------------
(['TA0008'], ['T1145', 'T1184'])
['TA0008'] ['T1145', 'T1184']
-----------------------------
(['TA0002', 'TA0005'], ['T1202', 'T1218', 'T1191'])
['TA0002', 'TA0005'] ['T1202', 'T1218', 'T1191']
-----------------------------
(['TA0006', 'TA0003', 'TA0004', 'TA0005', 'TA0007', 'TA0009', 'TA0011'], ['T1123', 'T1082', 'T1071', 'T1140', 'T1057', 'T1027', 'T1105'])
['TA0006', 'TA0003', 'TA0004', 'TA0005', 'TA0007', 'TA0009', 'TA0011'] ['T1123', 'T1082', 'T1071', 'T1140', 'T1057', 'T1027', 'T1105']
-----------------------------
(['TA0006', 'TA0008',

([], ['T1009', 'T1054'])
[] ['T1009', 'T1054']
-----------------------------
(['TA0002', 'TA0005'], ['T1063', 'T1117', 'T1127'])
['TA0002', 'TA0005'] ['T1063', 'T1117', 'T1127']
-----------------------------
(['TA0003'], [])
['TA0003'] []
-----------------------------
(['TA0006', 'TA0003', 'TA0005'], [])
['TA0006', 'TA0003', 'TA0005'] []
-----------------------------
(['TA0004', 'TA0005'], [])
['TA0004', 'TA0005'] []
-----------------------------
(['TA0003', 'TA0004', 'TA0005', 'TA0011'], ['T1156'])
['TA0003', 'TA0004', 'TA0005', 'TA0011'] ['T1156']
-----------------------------
(['TA0006', 'TA0002', 'TA0040', 'TA0005', 'TA0001'], ['T1192', 'T1193', 'T1064', 'T1204', 'T1027', 'T1081', 'T1086'])
['TA0006', 'TA0002', 'TA0040', 'TA0005', 'TA0001'] ['T1192', 'T1193', 'T1064', 'T1204', 'T1027', 'T1081', 'T1086']
-----------------------------
(['TA0004'], ['T1213'])
['TA0004', 'TA0009'] ['T1213']
-----------------------------
(['TA0003'], ['T1163', 'T1165', 'T1162', 'T1155'])
['TA0003'] ['T1

(['TA0002', 'TA0008', 'TA0005', 'TA0011'], [])
['TA0002', 'TA0008', 'TA0005', 'TA0011'] []
-----------------------------
(['TA0006', 'TA0009'], ['T1115'])
['TA0006', 'TA0009'] ['T1115']
-----------------------------
(['TA0007'], ['T1007'])
['TA0007'] ['T1007']
-----------------------------
(['TA0002', 'TA0003', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0011'], ['T1033', 'T1165', 'T1107', 'T1145', 'T1045', 'T1064', 'T1027', 'T1105'])
['TA0002', 'TA0003', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0011'] ['T1033', 'T1165', 'T1107', 'T1145', 'T1045', 'T1064', 'T1027', 'T1105']
-----------------------------
(['TA0006', 'TA0005', 'TA0009', 'TA0011'], [])
['TA0006', 'TA0005', 'TA0009', 'TA0011'] []
-----------------------------
(['TA0006', 'TA0002', 'TA0004', 'TA0008', 'TA0001'], ['T1192', 'T1193', 'T1045', 'T1210', 'T1110', 'T1081', 'T1086'])
['TA0006', 'TA0002', 'TA0004', 'TA0008', 'TA0001'] ['T1192', 'T1193', 'T1045', 'T1210', 'T1110', 'T1081', 'T1086']
-----------------------------
(['T

(['TA0002', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0007', 'TA0011'], ['T1047', 'T1033', 'T1040', 'T1071', 'T1106', 'T1112', 'T1016', 'T1027', 'T1493', 'T1022', 'T1105'])
['TA0002', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0007', 'TA0011'] ['T1047', 'T1033', 'T1071', 'T1106', 'T1112', 'T1016', 'T1027', 'T1493', 'T1022', 'T1105']
-----------------------------
(['TA0002', 'TA0003', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0011'], ['T1099', 'T1007', 'T1005', 'T1050', 'T1131', 'T1107', 'T1085', 'T1016', 'T1070', 'T1104', 'T1060', 'T1023', 'T1027', 'T1002', 'T1095', 'T1022'])
['TA0002', 'TA0003', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0011'] ['T1099', 'T1007', 'T1005', 'T1050', 'T1131', 'T1107', 'T1085', 'T1016', 'T1070', 'T1104', 'T1060', 'T1023', 'T1027', 'T1002', 'T1095', 'T1022']
-----------------------------
([], ['T1109', 'T1013', 'T1054', 'T1500'])
[] ['T1109', 'T1013', 'T1054', 'T1500']
-----------------------------
(['TA0040'], ['T1205', 'T1499'])
['TA0040', 'TA0003'] ['

([], [])
[] []
-----------------------------
(['TA0008'], ['T1021', 'T1175'])
['TA0008'] ['T1021', 'T1175']
-----------------------------
(['TA0006', 'TA0003', 'TA0001'], ['T1078'])
['TA0006', 'TA0003', 'TA0001', 'TA0004', 'TA0005'] ['T1078']
-----------------------------
(['TA0010', 'TA0001'], ['T1050', 'T1076', 'T1074', 'T1048', 'T1095', 'T1012', 'T1043'])
['TA0010', 'TA0001', 'TA0011'] ['T1050', 'T1076', 'T1074', 'T1048', 'T1095', 'T1012', 'T1043']
-----------------------------
(['TA0002', 'TA0003'], ['T1086'])
['TA0002', 'TA0003'] ['T1086']
-----------------------------
(['TA0009'], ['T1174'])
['TA0009'] ['T1174']
-----------------------------
(['TA0003'], ['T1058'])
['TA0003'] ['T1058']
-----------------------------
(['TA0007'], ['T1482'])
['TA0007'] ['T1482']
-----------------------------
(['TA0003'], ['T1215'])
['TA0003'] ['T1215']
-----------------------------
(['TA0006'], ['T1174'])
['TA0006'] ['T1174']
-----------------------------
([], [])
[] []
-----------------------------

(['TA0040'], ['T1491'])
['TA0040'] ['T1491']
-----------------------------
(['TA0004', 'TA0005'], ['T1055'])
['TA0004', 'TA0005'] ['T1055']
-----------------------------
(['TA0011'], ['T1483'])
['TA0011'] ['T1483']
-----------------------------
(['TA0011'], ['T1024'])
['TA0011'] ['T1024']
-----------------------------
(['TA0006'], ['T1187'])
['TA0006'] ['T1187']
-----------------------------
(['TA0002', 'TA0003', 'TA0004'], ['T1053', 'T1217'])
['TA0002', 'TA0003', 'TA0004'] ['T1053']
-----------------------------
([], [])
[] []
-----------------------------
(['TA0002', 'TA0005'], [])
['TA0002', 'TA0005'] []
-----------------------------
(['TA0005'], ['T1045', 'T1188', 'T1015', 'T1028'])
['TA0005'] ['T1045', 'T1188', 'T1015', 'T1028']
-----------------------------
(['TA0002', 'TA0003', 'TA0008', 'TA0005', 'TA0009', 'TA0011'], ['T1033', 'T1063', 'T1071', 'T1140', 'T1107', 'T1016', 'T1059', 'T1064', 'T1027', 'T1105'])
['TA0002', 'TA0003', 'TA0008', 'TA0005', 'TA0009', 'TA0011', 'TA0007'] 

(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0007', 'TA0011'], ['T1016', 'T1110', 'T1203', 'T1046'])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0007', 'TA0011'] ['T1016', 'T1110', 'T1203', 'T1046']
-----------------------------
(['TA0008'], ['T1021'])
['TA0008'] ['T1021']
-----------------------------
(['TA0040', 'TA0005'], [])
['TA0040', 'TA0005'] []
-----------------------------
([], [])
[] []
-----------------------------
(['TA0004', 'TA0005'], [])
['TA0004', 'TA0005'] []
-----------------------------
(['TA0004', 'TA0005'], ['T1181'])
['TA0004', 'TA0005'] ['T1181']
-----------------------------
(['TA0002', 'TA0008', 'TA0005', 'TA0011'], ['T1063', 'T1117', 'T1064', 'T1060', 'T1027', 'T1105'])
['TA0002', 'TA0008', 'TA0005', 'TA0011'] ['T1063', 'T1117', 'T1064', 'T1060', 'T1027', 'T1105']
-----------------------------
([], ['T1108'])
[] ['T1108']
-----------------------------
(['TA0006', 'TA0002', 'TA0003', 'TA0005', 'TA0007', 'TA0009', 'TA0011'], ['T1083', 'T1057', 'T1105'])
['TA0006

In [33]:
evaluation(Y_test_tactics_hanging_node, Y_test_tactic)

,metric,result
0,macro precision,0.646103
1,micro precision,0.684553
2,macro recall,0.665709
3,micro recall,0.688471
4,macro fscore,0.648513
5,micro fscore,0.685333


In [34]:
evaluation(Y_test_techniques_hanging_node, Y_test)

/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,metric,result
0,macro precision,0.602655
1,micro precision,0.491468
2,macro recall,0.387894
3,micro recall,0.331225
4,macro fscore,0.482797
5,micro fscore,0.448110


# Train Model on Reports Tactics:

In [ ]:
# see if training on only techniques improves or not
# only train techniques on reports belonging to a techniques' tactic

In [93]:
# Key as technique, value as tactic.

from collections import defaultdict
tactic_to_technique = defaultdict(list)
for tactic in tactic_data:
    for technique in tactic_data[tactic]['Technique_ID'][0]:
        if tactic in Y_train_tactic.columns:
            tactic_to_technique[technique].append(tactic)

In [62]:
Y = pd.concat([Y_train, Y_test]).reset_index(drop=True)
Y_tactic = pd.concat([Y_train_tactic, Y_test_tactic]).reset_index(drop=True)

In [94]:
# check for matching between tactic and technique
# find technique that are not in dictionary. 
# for Y_r, find report that only contains that technique and find what tactics is has 

for technique in Y.columns:
    if technique not in tactic_to_technique:
        index = Y[(Y.sum(axis=1)==1) & (Y[technique] == 1)].index[-1]
        tactics = [tactic for tactic in Y_tactic.columns if 
                       int(Y_tactic.loc[index, tactic])==1]
        if len(tactics) == 0:
            print(technique)
            print(Y[(Y.sum(axis=1)==1) & (Y[technique] == 1)].index)
        tactic_to_technique[technique] = tactics
    
    

In [96]:
Y_train_tactic[Y_train_tactic.sum(axis=1)==0]

,TA0006,TA0002,TA0040,TA0003,TA0004,TA0008,TA0005,TA0010,TA0007,TA0009,TA0011,TA0001
977,0,0,0,0,0,0,0,0,0,0,0,0
1090,0,0,0,0,0,0,0,0,0,0,0,0
1150,0,0,0,0,0,0,0,0,0,0,0,0
1265,0,0,0,0,0,0,0,0,0,0,0,0
1266,0,0,0,0,0,0,0,0,0,0,0,0
1267,0,0,0,0,0,0,0,0,0,0,0,0
1268,0,0,0,0,0,0,0,0,0,0,0,0
1345,0,0,0,0,0,0,0,0,0,0,0,0
1346,0,0,0,0,0,0,0,0,0,0,0,0
1347,0,0,0,0,0,0,0,0,0,0,0,0


In [83]:
X_train_text.iloc[977]

' APT   Exposing One of China s Cyber Espionage Units   Mandiant Skip to main content Mandiant has created a task force   initiated a Global Event to track the Russian invasion of Ukraine  Visit the Ukraine Crisis Resource Center to learn more    Platform Mandiant Advantage Multi vendor XDR platform that delivers expertise and frontline intelligence to security teams of all sizes  Platform overview Platform menu Automated Defense Rapid event investigation   remediation Attack Surface Management Fully map your entire environment Security Validation Validate controls are working properly Threat Intelligence Integrate latest intel from the frontlines Ransomware Defense Validation Test your ability to prevent ransomware Digital Threat Monitoring Visibility into the open  deep and dark web Managed Defense Amplify your team with Mandiant experts Get started for freeRegister for Mandiant Advantage Threat intelligence Solutions Mandiant Solutions Solve your toughest cyber security challenges w

In [61]:
Y_train = Y_train.reset_index(drop=True)
Y_train_tactic = Y_train_tactic.reset_index(drop=True)


In [56]:
Y_train_tactic

,TA0006,TA0002,TA0040,TA0003,TA0004,TA0008,TA0005,TA0010,TA0007,TA0009,TA0011,TA0001
0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,1,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,1,0,1,0,0,0,0,0
4,0,1,0,1,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2297,1,1,0,1,1,1,1,0,1,1,1,1
2298,0,1,0,1,1,0,1,0,0,0,1,0
2299,0,0,0,0,0,0,0,0,0,0,1,0
2300,0,1,0,1,0,0,1,0,1,1,1,0


In [57]:
X_train

,00,000,0000,00000,000000,00000000,0000000000,000000000000,0000000000000000,000000000046,...,zzzb,zzzd,zzzh,zzzj,zzzl,zzzz,zzzzco,zzzzk,zzzzzz,zzzzzzzzzzzzzzzz
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.009932,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2067,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2068,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2069,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2070,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
Y_train[Y_train['T1108']==1]

,T1066,T1047,T1156,T1113,T1067,T1037,T1033,T1003,T1129,T1492,...,T1124,T1035,T1086,T1490,T1216,T1094,T1043,T1211,T1127,T1077
421,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
678,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
872,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
877,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
902,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
929,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
954,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
977,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [92]:
tactic_to_technique["T1108"]

[]

In [95]:
classifiers = []
for technique in Y_train.columns:
    indexes = Y_train_tactic[Y_train_tactic.apply(lambda x: any(x[tactic] ==1 for tactic in tactic_to_technique[technique]), axis=1)].index
    X_train_technique = X_train.loc[indexes]
    Y_train_technique = Y_train.loc[indexes, technique]
    clf = LinearSVC(penalty = 'l2', loss = 'squared_hinge', dual = False, max_iter = 1000, class_weight = 'balanced', random_state=42)
    clf.fit(X_train_technique, Y_train_technique)
    classifiers.append(clf)
    

In [ ]:
Y_pred = pd.DataFrame(0, index=X_test.index, columns=Y_test.columns)
for i, technique in enumerate(Y_test.columns):
    indexes = Y_test_pred_tactic[Y_test_pred_tactic.apply(lambda x: any(x[tactic] ==1 for tactic in tactic_to_technique[technique]), axis=1)].index
    Y_pred.loc[indexes, technique] = classifiers[i].predict(X_test.loc[indexes])

In [201]:
evaluation(Y_pred, Y_test) # performance for technique 

/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


,metric,result
0,macro precision,0.499174
1,micro precision,0.505065
2,macro recall,0.341547
3,micro recall,0.401380
4,macro fscore,0.429969
5,micro fscore,0.480253


# Linking Model:

In [56]:
data = '../src/scraping/tactic_dataset.json'

with open(data) as file:
    tactic_data = json.load(file)

In [60]:
df_tactic = pd.DataFrame(tactic_data).transpose()


In [65]:
df_tactic['Technique_ID'] = df_tactic['Technique_ID'].apply(lambda x: x[0])

In [67]:
df_tactic.columns

Index(['Tactic_ID', 'Link', 'Tactic_Name', 'Description', 'Technique_ID'], dtype='object')

In [68]:
df_tactic = df_tactic[['Tactic_ID', 'Technique_ID']]

In [69]:
df_tactic

,Tactic_ID,Technique_ID
TA0043,TA0043,"[T1595, T1592, T1589, T1590, T1591, T1598, T15..."
TA0042,TA0042,"[T1583, T1586, T1584, T1587, T1585, T1588, T1608]"
TA0001,TA0001,"[T1189, T1190, T1133, T1200, T1566, T1091, T11..."
TA0002,TA0002,"[T1059, T1609, T1610, T1203, T1559, T1106, T10..."
TA0003,TA0003,"[T1098, T1197, T1547, T1037, T1176, T1554, T11..."
TA0004,TA0004,"[T1548, T1134, T1547, T1037, T1543, T1484, T16..."
TA0005,TA0005,"[T1548, T1134, T1197, T1612, T1622, T1140, T16..."
TA0006,TA0006,"[T1557, T1110, T1555, T1212, T1187, T1606, T10..."
TA0007,TA0007,"[T1087, T1010, T1217, T1580, T1538, T1526, T16..."
TA0008,TA0008,"[T1210, T1534, T1570, T1563, T1021, T1091, T10..."


In [ ]:
# technique model
# Open Pickle: 

with open('tactic_model.pickle', 'rb') as handle:
    tactic_model = pickle.load(handle)